In [1]:
from modelgym.models import CtBClassifier
from sklearn.datasets import make_classification
from modelgym.utils import XYCDataset
from modelgym.trainers import TpeTrainer
from modelgym.metrics import Accuracy, RocAuc
from modelgym.report import Report
from modelgym.utils import ModelSpace, process_model_spaces
from hyperopt.pyll.base import scope
from hyperopt import hp
from time import sleep

/home/igor/anaconda3/envs/yadro/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.4 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


Wonderland optimization


In [2]:
from modelgym.client import Client
import os

ip = "localhost:50051"
workingFolder = os.getenv("HOME")+"/repo-storage/test"
client = Client('localhost:50051', workingFolder)

In [ ]:
##Prepare Data
X, y = make_classification(n_samples=500, n_features=20, n_informative=10, n_classes=2)
dataset = XYCDataset(X, y)




In [ ]:
#Load data in Azure File Storage
dataPath = client.sendData(dataset)

In [ ]:
models = ModelSpace(CtBClassifier,
                    space={'learning_rate': hp.loguniform('learning_rate', -5, -1)},
                    space_update=True)
trainer = TpeTrainer(models)

##Cluster optimization
trainer.crossval_optimize_params(Accuracy(), dataPath, metrics=[Accuracy()], client=client)

In [ ]:
##Non-cluster optimization
trainer.crossval_optimize_params(Accuracy(), dataset, metrics=[Accuracy()])

In [ ]:
reporter = Report(trainer.get_best_results(), dataset, [Accuracy()])
reporter.print_all_metric_results()